In [1]:
import ollama

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama


In [4]:
from langchain.document_loaders import PyPDFLoader

# Load the PDF document
pdf_loader = PyPDFLoader("Admission_Requirement.pdf")
raw_documents = pdf_loader.load()


In [5]:
raw_documents
for raw_doc in raw_documents:
    print(raw_doc)

page_content='   
  Entry Requirements and Guidelines for Selecting an Undergraduate Programme   2024/2025   Published by The Academic Affairs Division Registrar’s Office April 2024             
' metadata={'source': 'Admission_Requirement.pdf', 'page': 0}
page_content='1  
Table of Contents 1.0 STEP-BY-STEP INSTRUCTIONS ..................................................................................... 2 A. STEP 1: Purchase of E-Voucher ................................................................................................ 2 B. STEP 2: Check List ................................................................................................................... 2 C. STEP 3 Application .................................................................................................................. 3 2.0 CHOICE OF PROGRAMME OF STUDY ........................................................................... 5 LIST OF UNDERGRADUATE PROGRAMMES OFFERED BY COLLEGES ..............

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documents = text_splitter.split_documents(raw_documents)

In [7]:
len(documents)

411

In [8]:
print(documents[0])

page_content='Entry Requirements and Guidelines for Selecting an Undergraduate Programme   2024/2025   Published by The Academic Affairs Division Registrar’s Office April 2024' metadata={'source': 'Admission_Requirement.pdf', 'page': 0}


In [9]:
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

C:\Users\User\AppData\Local\Temp\ipykernel_11720\3355370779.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")


In [10]:
db = Chroma.from_documents(documents, embedding=oembed)

In [14]:
query = "Entry requirement for BSc. Nursing"
docs = db.similarity_search(query)

In [15]:
len(docs)

4

In [16]:
print(docs[0].page_content)

Technology will be placed in SECOND YEAR. All Candidates must have TWO (2) YEARS post-qualification experience and must have passed an interview.  •  Short-listed applicants may take an Entrance Examination and pass an interview. E. BSc. GENRAL NURSING (MATURE APPLICANTS FOR TOP-UP) Entry


In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [18]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [19]:
model = ChatOllama(
    model="llama3.2:1b",
    temperature=0
    
)

In [20]:
retriever = db.as_retriever()

In [21]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [22]:
chain = (
    {"context":retriever | format_docs, "question": RunnablePassthrough()}
    |prompt
    |model
    |StrOutputParser()
)

In [26]:
chain.invoke("list the programms in COLLEGE OF HUMANITIES AND SOCIAL SCIENCES")

'Here are the programs offered by College of Humanities and Social Sciences:\n\n1. BA. Akan Language and Culture\n2. BA. Economics\n3. BA. English\n4. BA. French and Francophone Studies\n5. BA. Geography and Rural Development\n6. BA. History'